In [ ]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings

In [11]:
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv(r'C:\kartheek\Krish\notebook\data\stud.csv')
warnings.filterwarnings('ignore')

In [8]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [9]:
x = df.drop(columns=['math_score'], axis=1)
y= df['math_score']

In [13]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [16]:
num_features = x.select_dtypes(exclude=['object']).columns
cat_features = x.select_dtypes(include=['object']).columns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

Preproecssor = ColumnTransformer(
    [
        ("oneHotEncoder", oh_transformer, cat_features),
        ("standardScaler", numeric_transformer, num_features),
    ]
)

In [26]:
X= Preproecssor.fit_transform(x)
X.shape

(1000, 19)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape,X_test.shape

((800, 19), (200, 19))

In [28]:
#model selection
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square
    

In [29]:
models = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "KNN": KNeighborsRegressor(),
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
}

In [36]:
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(f"Model: {list(models.keys())[i]}")
    model_list.append(list(models.keys())[i])

    print(f"Train R2 Score: {model_train_r2}")
    print(f"Test mean squared error: {model_train_rmse}")
    print(f"mean absolute error: {model_train_mae}")

    print("--------------------------------")

    print(f"Test R2 Score: {model_test_r2}")
    print(f"Test mean squared error: {model_test_rmse}")
    print(f"mean absolute error: {model_test_mae}")

    r2_list.append(model_test_r2)

    print("=================================")
    print("\n")



Model: Random Forest
Train R2 Score: 0.9760574073706877
Test mean squared error: 2.323316017054969
mean absolute error: 1.8351739583333333
--------------------------------
Test R2 Score: 0.8516071044936233
Test mean squared error: 6.009133861372111
mean absolute error: 4.664641666666666


Model: Decision Tree
Train R2 Score: 0.9996534669718089
Test mean squared error: 0.2795084971874737
mean absolute error: 0.01875
--------------------------------
Test R2 Score: 0.7381629861953559
Test mean squared error: 7.9821676254010105
mean absolute error: 6.335


Model: KNN
Train R2 Score: 0.8554978341651085
Test mean squared error: 5.707683417990174
mean absolute error: 4.516749999999999
--------------------------------
Test R2 Score: 0.7838129945787431
Test mean squared error: 7.253040741647602
mean absolute error: 5.621


Model: Linear Regression
Train R2 Score: 0.8743172040139593
Test mean squared error: 5.323050852720514
mean absolute error: 4.266711846071957
--------------------------------

In [37]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2 Score']).sort_values(by='R2 Score', ascending=False)

,Model Name,R2 Score
4,Ridge,0.880593
3,Linear Regression,0.880433
8,CatBoosting Regressor,0.851632
0,Random Forest,0.851607
9,AdaBoost Regressor,0.847623
7,XGBRegressor,0.827797
5,Lasso,0.825320
2,KNN,0.783813
1,Decision Tree,0.738163
6,SVR,0.728600


In [ ]:
lin_model = LinearRegression(fit_intercept=True)